In [1]:
import numpy as np
import random
import pandas as pd 
from copy import deepcopy
import math
import time
from math import ceil,floor

In [2]:
def g1(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    x5=x[4]
    return 85.334407 + 0.0056858*x2*x5 + 0.0006262*x1*x4 - 0.0022053*x3*x5

In [3]:
def g2(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    x5=x[4]
    
    return 80.51249 + 0.0071317*x2*x5 + 0.0029955*x1*x2 - 0.0021813*x3**2

In [4]:
def g3(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    x5=x[4]

    return 9.300961 + 0.0047026*x3*x5 + 0.0012547*x1*x3 + 0.0019085*x3*x4

In [5]:
def is_feasable(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    x5=x[4]
    if x1<78  or x1>102:
        #print("x1")
        return False
    if x2<33 or x2>45:
        #print("x2")
        return False
        
    if x3<27 or x3>45:
        #print("x3")
        return False
    if x4<27 or x4>45:
        #print("x4")
        return False
    if x5<27 or x5>45:
        #print("x5")
        return False

    g1res=g1(x)
    g2res=g2(x)
    g3res=g3(x)

    if g1res<0 or g1res>92:
        #print("g1")
        return False
    if g2res<90 or g2res>110:
        #print("g2")
        return False
    if g3res<20 or g3res>25:
        #print("g3")
        return False
    
    return True
    

In [6]:
def himmelblau(x):
    x3=x[2]
    x1=x[0]
    x5=x[4]
    toReturn= -(5.3578547*x3**2 + 0.8356891*x1*x5 + 37.293239*x1 - 40792.141)
    g1res=g1(x)
    if g1res>92:
        toReturn+=-(ceil(g1res)-92)*10000
    elif g1res<0:
        toReturn+=floor(g1res)*10000
    
    g2res=g2(x)
    if g2res>110:
        toReturn-=(ceil(g2res)-110)*10000
    elif g2res<90:
        toReturn-=(90-floor(g2res))*10000
    
    g3res=g3(x)
    if g3res>25:
        toReturn-=(ceil(g3res)-25)*10000
    elif g3res<20:
        toReturn-=(20-floor(g3res))*10000
    return toReturn

In [7]:
def actual_himmelblau(x):
    x3=x[2]
    x1=x[0]
    x5=x[4]
    return -(5.3578547*x3**2 + 0.8356891*x1*x5 + 37.293239*x1 - 40792.141)

In [8]:
class Individual:
    global_best_position=None
    global_best_fitness=None
    def __init__(self,bounds,velocity_calc,calc_fitness,age=0,life_expectancy=120):
        self.lower_bounds = np.array([bound[0] for bound in bounds])
        self.upper_bounds = np.array([bound[1] for bound in bounds])        

        self.life_expectancy=life_expectancy
        
        self.position = np.random.uniform(self.lower_bounds, self.upper_bounds, len(bounds))
        self.velocity = np.random.uniform(self.lower_bounds - self.upper_bounds,
                                          self.upper_bounds - self.lower_bounds,
                                          len(bounds))
        self.calc_fitness=calc_fitness
        self.fitness=calc_fitness(self.position)
        self.age=age

        self.velocity_calc=velocity_calc
        
        self.personal_best_fitness=self.fitness
        self.personal_best_position=self.position.copy()
        
        if Individual.global_best_fitness is None or Individual.global_best_fitness<self.fitness:
                Individual.global_best_fitness=self.fitness
                Individual.global_best_position=self.position.copy()
    
        
    def update_position(self):
        self.position = np.clip(self.position + self.velocity,
                                self.lower_bounds,
                                self.upper_bounds)
        self.update_fitness()
       
                
    def update_velocity(self):
        personal_direction=(self.personal_best_position - self.position)
        global_direction=(Individual.global_best_position - self.position)
        
        #if any([x!=0 for x in personal_direction]):
         #   personal_direction/=np.linalg.norm(personal_direction)
        #if any([x!=0 for x in global_direction]):
         #   global_direction/=np.linalg.norm(global_direction)
        

        
        cognitive_velocity =random.random() * self.velocity_calc["cognitive"](self.age,self.life_expectancy)*personal_direction#*np.linalg.norm(self.velocity) 
        social_velocity =random.random() * self.velocity_calc["group"](self.age,self.life_expectancy)* global_direction#*np.linalg.norm(self.velocity)
        inertia = self.velocity_calc["inertia"](self.age,self.life_expectancy) * self.velocity
        self.velocity = inertia + cognitive_velocity + social_velocity
        self.velocity=np.clip(self.velocity,2*self.lower_bounds,2*self.upper_bounds)
    def update_fitness(self):
        self.fitness=self.calc_fitness(self.position)
        if self.personal_best_fitness is None or self.fitness>self.personal_best_fitness:
            self.personal_best_fitness=self.fitness
            self.personal_best_position=self.position.copy()
            if Individual.global_best_fitness is None or Individual.global_best_fitness<self.fitness:
                Individual.global_best_fitness=self.fitness
                Individual.global_best_position=self.position.copy()
    def mutate(self,mut_prob):
        #TODO
        for i in range(len(self.position)):
            if random.random()<mut_prob:
                self.position[i]+=0.01*random.uniform(self.lower_bounds[i],self.upper_bounds[i])
            if random.random()<mut_prob:
                self.velocity[i]+=0.01*random.uniform(self.lower_bounds[i],self.upper_bounds[i])
        self.position = np.clip(self.position + self.velocity,
                                self.lower_bounds,
                                self.upper_bounds)

In [9]:
def crossover(p1:Individual,p2:Individual,c1:Individual,c2:Individual):
    coef=random.random()
    c1.position=coef*p1.position + (1-coef)*p2.position
    c1.velocity=(coef*p1.velocity + (1-coef)*p2.velocity)/2

    c2.position=coef*p2.position +(1-coef)*p1.position
    c2.velocity=(coef*p2.velocity +(1-coef)*p1.velocity)/2

    c1.age=0
    c2.age=0

In [10]:
def mutation(ind:Individual,mut_prob=0.01):
    #ind.mutate(mut_prob)
    pass

In [11]:
def selection(population,tournament_size):
    allowed=random.sample(population,tournament_size)
    return max(allowed,key=lambda x:x.fitness)

In [12]:
velocity_calculator={
    "cognitive": lambda age,life_expectancy:2*(1-age/life_expectancy),
    "group" : lambda age ,life_expectancy:2*(age/life_expectancy),
    "inertia" : lambda age , life_expectancy:(120/120)
}
vezbe_velocity_calc={
    "cognitive":lambda age,life_expectancy:0.7 ,
    "group":lambda age, life_expectancy:1.0 ,
    "inertia":lambda age ,life_expectancy:1.0
    
}
maxtime=10**(-1)
def psogahybrid(pop_size, num_dimensions, allowed_time,velocity_calc,fitness_calc,life_expectancy=120,tournament_size=4):
    bounds = [(78, 102),(33,45),(27,45),(27,45),(27,45)]
    
    population = [Individual(bounds=bounds,
                      calc_fitness=fitness_calc,life_expectancy=life_expectancy,velocity_calc=velocity_calc,age=i%life_expectancy
                      ) for i in range(pop_size)]
    #print(Individual.global_best_fitness)
    
    
    start=time.time()
    while True:
        #sorted(population,key=lambda x:-x.fitness)
        
        for i in range(len(population)):
               
            population[i].update_velocity()
            population[i].update_position()
            population[i].age+=1
            population[i].update_fitness()
            
            if population[i].age==life_expectancy:
                     
                toPairWith=selection(population,tournament_size)
                child1=Individual(bounds=bounds,calc_fitness=fitness_calc,velocity_calc=velocity_calc,life_expectancy=life_expectancy)
                child2=Individual(bounds=bounds,calc_fitness=fitness_calc,velocity_calc=velocity_calc,life_expectancy=life_expectancy)
                crossover(population[i],toPairWith,child1,child2)
             
                mutation(child1)
                mutation(child2)
                population[i]=max([child1,child2],key=lambda x:x.fitness)
                population[i].update_fitness()
        if time.time()-start>allowed_time:
                tmp=Individual.global_best_position
                #if not is_feasable(Individual.global_best_position):
                 
                #print("not feasible")
                Individual.global_best_fitness=None
                Individual.global_best_position=None
                
                return tmp
       

   

In [13]:
#nasa hibridna optimizacija
psogahybrid(pop_size=50, num_dimensions=5, allowed_time=0.1,velocity_calc=vezbe_velocity_calc,life_expectancy=10,fitness_calc=himmelblau,tournament_size=10)


array([84.69524341, 33.7895484 , 30.49067006, 40.47343616, 36.07587421])

In [14]:
#goal driven generational learning

Ideja algoritma:
    Imamo populaciju jedinki koje imaju svoju poziciju u prostoru i svoje usmerenje
    Imaju age-broj iteracija koliko je jedinka ziva.
    Imaju life_expectancy-zivotni vek u broju iteracija
    Pusticemo svaku jedinku da se krece kroz prostor nalik na pso dok ne bude imala 
    life expectancy godina.
    Razlika u odnosu na pso je sto dinamicki racunamo koeficijente prema broju godina i
    zivotnog ocekivanja.
    Kada jedinka dostigne life expectancy godina koristimo tournament selection da bi odabrali jedinku
    sa kojom cemo da je ukrstimo kako bi je zamenila u populaciji.(od dva deteta biramo najbolje)
    to radimo dok ne prodjemo maximalni broj iteracija.
    Mozda bi mutacija dovela do prevelikog diverziteta.
    Mozda bi neki elitism doprineo.
    

In [15]:
times=[10**(-3),10**(-2),10**(-1),1]
num_trys=1000

In [16]:
times=[10**(-3),10**(-2),10**(-1),1]
num_trys=1000
print("Function : Himbelblau")
print("Swarm size: " + "50")
print("Num dimensions: " + "5")
print("c_i: "+"0.7")
print("c_p: " + "1.0")
print("c_g: " + "1.0")
print("ga_min_num : 10")
print("ga_max_num :50")
print("ga_min_ps : 50")
print("ga_max_ps : 100")
print("ga_min_iter : 20")
print("ga_max_iter : 100")
print("y : 2")
print("b :3")
for allowed_time in times:
    suma=0
    counter=0
    for _ in range(num_trys):
        position=psogahybrid(pop_size=50, num_dimensions=5, allowed_time=allowed_time,velocity_calc=vezbe_velocity_calc,life_expectancy=30,fitness_calc=himmelblau,tournament_size=10)
        
        if is_feasable(position):
            suma+=actual_himmelblau(position)
            counter+=1
    suma=-suma
    suma/=counter
    
    print("    Time : " + str(allowed_time))
    print("    NumHits : "+ str(counter) + "/" + str(num_trys))
    print("    Average minimum: " +str(suma))
    print("\n")

Function : Himbelblau
Swarm size: 50
Num dimensions: 5
c_i: 0.7
c_p: 1.0
c_g: 1.0
ga_min_num : 10
ga_max_num :50
ga_min_ps : 50
ga_max_ps : 100
ga_min_iter : 20
ga_max_iter : 100
y : 2
b :3
    Time : 0.001
    NumHits : 1000/1000
    Average minimum: -29031.514410706855




KeyboardInterrupt: 

In [19]:
suma=0
counter=0
for _ in range(50):
    position=psogahybrid(pop_size=50, num_dimensions=5, allowed_time=50,velocity_calc=vezbe_velocity_calc,life_expectancy=20,fitness_calc=himmelblau,tournament_size=10)
        
    if is_feasable(position):
        suma+=actual_himmelblau(position)
        counter+=1
suma=-suma
suma/=counter
    
print("    Time : " + str(50))
print("    NumHits : "+ str(counter) + "/" + str(50))
print("    Average minimum: " +str(suma))
print("\n")

    Time : 50
    NumHits : 50/50
    Average minimum: -30427.684853435578


